<a href="https://colab.research.google.com/github/Kavi-Dew-23/4338-Dewapura/blob/main/Copy_of_Test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder

In [3]:
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")
print(df_train.head())


   Unnamed: 0      id  Gender      Customer Type  Age   Type of Travel  \
0           0   70172    Male     Loyal Customer   13  Personal Travel   
1           1    5047    Male  disloyal Customer   25  Business travel   
2           2  110028  Female     Loyal Customer   26  Business travel   
3           3   24026  Female     Loyal Customer   25  Business travel   
4           4  119299    Male     Loyal Customer   61  Business travel   

      Class  Flight Distance  Inflight wifi service  \
0  Eco Plus              460                      3   
1  Business              235                      3   
2  Business             1142                      2   
3  Business              562                      2   
4  Business              214                      3   

   Departure/Arrival time convenient  ...  Inflight entertainment  \
0                                  4  ...                       5   
1                                  2  ...                       1   
2                

In [5]:
df_train.drop(columns=["Unnamed: 0", "id"], inplace=True)

In [4]:
# Drop NaN rows
df_train.dropna(inplace=True)

In [6]:
# Check for null values
print(df_train.isnull().sum())

Gender                               0
Customer Type                        0
Age                                  0
Type of Travel                       0
Class                                0
Flight Distance                      0
Inflight wifi service                0
Departure/Arrival time convenient    0
Ease of Online booking               0
Gate location                        0
Food and drink                       0
Online boarding                      0
Seat comfort                         0
Inflight entertainment               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Inflight service                     0
Cleanliness                          0
Departure Delay in Minutes           0
Arrival Delay in Minutes             0
satisfaction                         0
dtype: int64


In [7]:
#drop missing values
df_train.dropna(inplace=True)

In [8]:
print(df_train.head())

   Gender      Customer Type  Age   Type of Travel     Class  Flight Distance  \
0    Male     Loyal Customer   13  Personal Travel  Eco Plus              460   
1    Male  disloyal Customer   25  Business travel  Business              235   
2  Female     Loyal Customer   26  Business travel  Business             1142   
3  Female     Loyal Customer   25  Business travel  Business              562   
4    Male     Loyal Customer   61  Business travel  Business              214   

   Inflight wifi service  Departure/Arrival time convenient  \
0                      3                                  4   
1                      3                                  2   
2                      2                                  2   
3                      2                                  5   
4                      3                                  3   

   Ease of Online booking  Gate location  ...  Inflight entertainment  \
0                       3              1  ...                

In [9]:
# List of columns to encode
label_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# Save encoders for each categorical column
encoders = {}

# Encode each column
for col in label_cols:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col])
    encoders[col] =le

print(df_train.head(5))


   Gender  Customer Type  Age  Type of Travel  Class  Flight Distance  \
0       1              0   13               1      2              460   
1       1              1   25               0      0              235   
2       0              0   26               0      0             1142   
3       0              0   25               0      0              562   
4       1              0   61               0      0              214   

   Inflight wifi service  Departure/Arrival time convenient  \
0                      3                                  4   
1                      3                                  2   
2                      2                                  2   
3                      2                                  5   
4                      3                                  3   

   Ease of Online booking  Gate location  ...  Inflight entertainment  \
0                       3              1  ...                       5   
1                       3           

In [10]:
#encode the target column
target_encoder = LabelEncoder()
df_train['satisfaction'] = target_encoder.fit_transform(df_train['satisfaction'])

In [11]:
print(df_train.head(5))

   Gender  Customer Type  Age  Type of Travel  Class  Flight Distance  \
0       1              0   13               1      2              460   
1       1              1   25               0      0              235   
2       0              0   26               0      0             1142   
3       0              0   25               0      0              562   
4       1              0   61               0      0              214   

   Inflight wifi service  Departure/Arrival time convenient  \
0                      3                                  4   
1                      3                                  2   
2                      2                                  2   
3                      2                                  5   
4                      3                                  3   

   Ease of Online booking  Gate location  ...  Inflight entertainment  \
0                       3              1  ...                       5   
1                       3           

In [12]:
# Generate synthetic complaint text

def generate_complaint(row):
    complaint = []
    if row['Inflight wifi service'] <= 2:
        complaint.append("wifi was slow and unstable.")
    if row['Food and drink'] <= 2:
        complaint.append("The food and drink were poor.")
    if row['Departure/Arrival time convenient'] <= 2:
        complaint.append("Departure and arrival time were very inconvienient.")
    if row['Ease of Online booking'] <= 2:
        complaint.append("Online booking was difficult.")
    if row['Seat comfort'] <= 2:
        complaint.append("Seat was uncomfortable.")
    if row['Gate location'] <= 2:
        complaint.append("gate location was confusing.")
    if row['Inflight entertainment'] <= 2:
        complaint.append("inflight entertaintment was poor.")
    if row['On-board service'] <= 2:
        complaint.append("Onboard service felt unhelpful.")
    if row["Leg room service"] <= 2:
        complaint.append("Legroom was insufficient.")
    if row['Baggage handling'] <= 2:
        complaint.append("Baggage handling was unsatisfactory.")
    if row['Checkin service'] <= 2:
        complaint.append("Check-in service was poor.")
    if row['Inflight service'] <= 2:
        complaint.append("Inflight service was below expectation.")
    if row['Cleanliness'] <= 2:
        complaint.append("The plane was not clean.")
    return " and ".join(complaint) if complaint else "No complaint"

df_train['Complaint_Text'] = df_train.apply(generate_complaint, axis=1)

In [13]:
# generate synthetic positive feedback
def generate_positive_feedback(row):
    feedback = []
    if row['Inflight wifi service'] >= 2:
        feedback.append("WiFi was fast and stable.")
    if row['Food and drink'] >= 2:
        feedback.append("The food and drink were delicious.")
    if row['Departure/Arrival time convenient'] >= 2:
        feedback.append("Departure and arrival time were very accurate.")
    if row['Ease of Online booking'] >= 2:
        feedback.append("Online booking was smooth.")
    if row['Seat comfort'] >= 2:
        feedback.append("Seats were very comfortable.")
    if row['Gate location'] >= 2:
        feedback.append("gate location was helpful.")
    if row['Inflight entertainment'] >= 2:
        feedback.append("Excellent inflight entertainment options.")
    if row['On-board service'] >= 2:
        feedback.append("The onboard service was excellent.")
    if row["Leg room service"] >= 2:
        feedback.append("Legroom was sufficient.")
    if row['Baggage handling'] >= 2:
        feedback.append("Baggage handling was satisfactory.")
    if row['Checkin service'] >= 2:
        feedback.append("Check-in process was smooth and quick.")
    if row['Inflight service'] >= 2:
        feedback.append("Inflight service was amazing.")
    if row['Cleanliness'] >= 2:
        feedback.append("The cabin was clean and well maintained.")
    return " ".join(feedback) if feedback else "No feedback"

df_train['Positive_Feedback'] = df_train.apply(generate_positive_feedback,axis=1)


In [14]:
df_train["Feedback_Text"] = df_train.apply(
    lambda row: row["Complaint_Text"] if row["satisfaction"] == 0 else row["Positive_Feedback"],
    axis=1
)

In [15]:
print(df_train.columns)

Index(['Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction', 'Complaint_Text', 'Positive_Feedback', 'Feedback_Text'],
      dtype='object')


In [16]:
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [17]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r"[^a-zA-Z]", " ", text.lower())
    words = text.split()
    cleaned = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(cleaned)

df_train["Cleaned_Text"] = df_train["Feedback_Text"].apply(clean_text)
print(df_train["Cleaned_Text"])

0                                   gate location confusing
1         food drink poor departure arrival time inconvi...
2         wifi fast stable food drink delicious departur...
3         wifi slow unstable food drink poor seat uncomf...
4         wifi fast stable food drink delicious departur...
                                ...                        
103899    wifi slow unstable food drink poor departure a...
103900    wifi fast stable food drink delicious departur...
103901    wifi slow unstable departure arrival time inco...
103902    wifi slow unstable food drink poor departure a...
103903    wifi slow unstable food drink poor seat uncomf...
Name: Cleaned_Text, Length: 103594, dtype: object


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_text = tfidf.fit_transform(df_train["Cleaned_Text"])


In [19]:
# Get feature (word) names
feature_names = tfidf.get_feature_names_out()

# Choose how many documents to print
num_docs_to_print = 5  # You can increase this

for i in range(num_docs_to_print):
    print(f"\n--- Document {i+1} ---")
    print("Cleaned Text:", df_train["Cleaned_Text"].iloc[i])

    # Get the tf-idf vector for document i
    row = X_text[i]

    # Convert it to a dense format and zip with feature names
    scores = zip(feature_names, row.toarray()[0])

    # Filter non-zero scores and sort by importance
    sorted_scores = sorted([(word, score) for word, score in scores if score > 0], key=lambda x: -x[1])

    print("TF-IDF Scores:")
    for word, score in sorted_scores:
        print(f"{word}: {score:.4f}")



--- Document 1 ---
Cleaned Text: gate location confusing
TF-IDF Scores:
confusing: 0.7595
gate: 0.4600
location: 0.4600

--- Document 2 ---
Cleaned Text: food drink poor departure arrival time inconvienient seat uncomfortable inflight entertaintment poor onboard service felt unhelpful check service poor plane clean
TF-IDF Scores:
poor: 0.5866
inconvienient: 0.2562
felt: 0.2551
unhelpful: 0.2551
uncomfortable: 0.2531
service: 0.2529
plane: 0.2422
entertaintment: 0.2373
arrival: 0.1587
departure: 0.1587
time: 0.1587
check: 0.1484
onboard: 0.1455
seat: 0.1455
clean: 0.1416
drink: 0.1411
food: 0.1411
inflight: 0.1317

--- Document 3 ---
Cleaned Text: wifi fast stable food drink delicious departure arrival time accurate online booking easy seat comfortable gate location helpful great inflight entertainment option onboard service excellent legroom sufficient baggage handling satisfactory check process smooth quick inflight service great cabin clean well maintained
TF-IDF Scores:
great: 0.31

In [20]:
#fine tuning DistilBERT
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# Create a dataset
#df_small = df_train.sample(n=5000, random_state=42).reset_index(drop=True)
df_small = df_train.copy()
df_nlp = df_small[['Cleaned_Text', 'satisfaction']].rename(columns={'Cleaned_Text': 'text', 'satisfaction': 'label'})

#convert pandas dataframe into Hugging face dataset
dataset = Dataset.from_pandas(df_nlp)

#Split the dataset into train and test
dataset = dataset.train_test_split(test_size=0.2)

In [21]:
# Tokenize the text (DistilBERT can't read raw text it needs tokenized inputs(word embedding))
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/82875 [00:00<?, ? examples/s]

Map:   0%|          | 0/20719 [00:00<?, ? examples/s]

In [22]:
#Load the pre trained DistilBERT for classification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Train the BERT model

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
)

In [24]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00


In [25]:
#evaluate the model performance during training and testing
import evaluate

#load accuracy metric from hugging face evaluate library
metric = evaluate.load("accuracy")

def compute_metric(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions,references=labels)

In [26]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metric,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dewapuraka (dewapuraka-university-of-ruhuna) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.000000,0.000000,1.000000
2,0.000000,0.000000,1.000000
3,0.000000,0.000000,1.000000


TrainOutput(global_step=15540, training_loss=0.0006796498582654885, metrics={'train_runtime': 3109.9744, 'train_samples_per_second': 79.944, 'train_steps_per_second': 4.997, 'total_flos': 8233676747712000.0, 'train_loss': 0.0006796498582654885, 'epoch': 3.0})

In [27]:
trainer.evaluate()

{'eval_loss': 5.753621537540887e-12,
 'eval_accuracy': 1.0,
 'eval_runtime': 79.6245,
 'eval_samples_per_second': 260.209,
 'eval_steps_per_second': 16.264,
 'epoch': 3.0}

In [28]:
df_train["satisfaction"].value_counts(normalize=True)

,proportion
satisfaction,
0,0.566606
1,0.433394


In [30]:
# Detect device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to the correct device

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [31]:
# Save the models

trainer.save_model("./saved_distilbert_model")
tokenizer.save_pretrained("./saved_distilbert_model")

('./saved_distilbert_model/tokenizer_config.json',
 './saved_distilbert_model/special_tokens_map.json',
 './saved_distilbert_model/vocab.txt',
 './saved_distilbert_model/added_tokens.json')

In [32]:
# To reload the saved model

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("./saved_distilbert_model")
tokenizer = DistilBertTokenizer.from_pretrained("./saved_distilbert_model")

In [34]:
# Custom examples to test model behavior
test_texts = [
    "The seat was very uncomfortable and the food was terrible.",
    "Everything was excellent, the flight was smooth and the staff were great.",
    "WiFi was bad and boarding took too long.",
    "Very friendly staff and excellent inflight entertainment.",
    "Seat was very comfortable and has very great inflight experience"
]

# Tokenize
inputs = tokenizer(test_texts, return_tensors="pt", padding=True, truncation=True)

# Predict
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# Decode label
predicted_labels = target_encoder.inverse_transform(predictions.numpy())
for text, label in zip(test_texts, predicted_labels):
    print(f"\nText: {text}\nPredicted: {label}")



Text: The seat was very uncomfortable and the food was terrible.
Predicted: neutral or dissatisfied

Text: Everything was excellent, the flight was smooth and the staff were great.
Predicted: neutral or dissatisfied

Text: WiFi was bad and boarding took too long.
Predicted: neutral or dissatisfied

Text: Very friendly staff and excellent inflight entertainment.
Predicted: satisfied

Text: Seat was very comfortable and has very great inflight experience
Predicted: satisfied
